In [1]:
import pandas as pd
df= pd.read_csv('../data/olist_dataset.csv')


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119143 entries, 0 to 119142
Data columns (total 42 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   order_id                       119143 non-null  object 
 1   customer_id                    119143 non-null  object 
 2   order_status                   119143 non-null  object 
 3   order_purchase_timestamp       119143 non-null  object 
 4   order_approved_at              118966 non-null  object 
 5   order_delivered_carrier_date   117057 non-null  object 
 6   order_delivered_customer_date  115722 non-null  object 
 7   order_estimated_delivery_date  119143 non-null  object 
 8   review_id                      118146 non-null  object 
 9   review_score                   118146 non-null  float64
 10  review_comment_title           13989 non-null   object 
 11  review_comment_message         50245 non-null   object 
 12  review_creation_date          

### Top Categories

In [4]:
top_categories = df['product_category_name'].value_counts().head(5)
print("Top 5 Product Categories by Number of Items Sold:\n")
print(top_categories.to_string())


Top 5 Product Categories by Number of Items Sold:

cama_mesa_banho           11988
beleza_saude              10032
esporte_lazer              9004
moveis_decoracao           8832
informatica_acessorios     8150


### Mean delivery time

In [5]:
 
df['delivery_time'] = pd.to_datetime(df.order_delivered_customer_date )- pd.to_datetime(df['order_purchase_timestamp'])
df['delivery_time'].mean()

Timedelta('12 days 11:42:49.317009730')

### payment types


In [6]:
df['payment_type'].value_counts()

credit_card    87776
boleto         23190
voucher         6465
debit_card      1706
not_defined        3
Name: payment_type, dtype: int64

### Total amount spent per order by each customer

In [7]:

order_totals = df.groupby(['customer_id', 'order_id'])['price'].sum().reset_index()
#order_totals.rename(columns={'price': 'total_spent'}, inplace=True)
customer_information = pd.merge(order_totals,df,on='order_id',how='left')
# Display the first rows


### Top 10 most active cities by number of orders


In [8]:
top_cities = df['customer_city'].value_counts().head(10)

print("Top 10 most active customer cities by number of orders:\n")
print(top_cities.reset_index().rename(columns={
    'index': 'customer_city',
    'customer_city': 'number_of_orders'
}))


Top 10 most active customer cities by number of orders:

           customer_city  number_of_orders
0              sao paulo             18875
1         rio de janeiro              8311
2         belo horizonte              3299
3               brasilia              2500
4               curitiba              1827
5               campinas              1757
6           porto alegre              1675
7               salvador              1544
8              guarulhos              1415
9  sao bernardo do campo              1131


### Distribution of review scores for customers from São Paulo (SP)


In [9]:
sp_review_distribution = df[df['customer_state'] == 'SP']['review_score'].value_counts().sort_index(ascending=False)

print("Review score distribution for customers in São Paulo (SP):\n")
print(sp_review_distribution.reset_index().rename(columns={
    'index': 'review_score',
    'review_score': 'number_of_reviews'
}))


Review score distribution for customers in São Paulo (SP):

   review_score  number_of_reviews
0           5.0              29311
1           4.0               9417
2           3.0               4034
3           2.0               1625
4           1.0               5535


### Average review score by customer state 


In [10]:
avg_score_by_state = (
    df.groupby('customer_state')['review_score']
    .mean()
    .sort_values(ascending=False)
    .reset_index()
    .rename(columns={'customer_state': 'State', 'review_score': 'Average Review Score'})
)

print("Average Review Score by State:\n")
print(avg_score_by_state)


Average Review Score by State:

   State  Average Review Score
0     AP              4.240964
1     TO              4.138643
2     SP              4.108609
3     AC              4.094737
4     PR              4.087130
5     AM              4.070175
6     MG              4.068543
7     MS              4.054651
8     RN              4.049296
9     RS              4.027184
10    PB              3.992101
11    SC              3.988861
12    DF              3.987981
13    RO              3.979381
14    GO              3.975710
15    ES              3.974656
16    PE              3.957469
17    MT              3.890277
18    PI              3.849123
19    CE              3.810167
20    BA              3.807235
21    RJ              3.799830
22    SE              3.795511
23    PA              3.786101
24    AL              3.715835
25    MA              3.670588
26    RR              3.576923


### Number of orders per month


In [11]:
df['shipping_month'] = pd.to_datetime(df['shipping_limit_date']).dt.to_period('M')

orders_per_month = (
    df['shipping_month']
    .value_counts()
    .sort_index()
    .reset_index()
    .rename(columns={'index': 'Month', 'shipping_month': 'Number of Orders'})
)

print("Number of Orders per Month:\n")
print(orders_per_month)


Number of Orders per Month:

      Month  Number of Orders
0   2016-09                 4
1   2016-10               387
2   2016-12                 1
3   2017-01               729
4   2017-02              1976
5   2017-03              2956
6   2017-04              2524
7   2017-05              4438
8   2017-06              4083
9   2017-07              4415
10  2017-08              5395
11  2017-09              5008
12  2017-10              5500
13  2017-11              7716
14  2017-12              8080
15  2018-01              7873
16  2018-02              7720
17  2018-03              9149
18  2018-04              7929
19  2018-05              9104
20  2018-06              7181
21  2018-07              6940
22  2018-08              9093
23  2018-09               105
24  2020-02                 2
25  2020-04                 2


### Top sellers by number of sales (number of orders handled)


In [12]:

top_sellers = df.groupby('seller_id')['order_id'].count().sort_values(ascending=False)

# Convert to DataFrame for better display and name columns
top_sellers_df = top_sellers.reset_index()
top_sellers_df.columns = ['Seller ID', 'Number of Sales']
print(top_sellers_df)


                             Seller ID  Number of Sales
0     4a3ca9315b744ce9f8e9374361493884             2155
1     6560211a19b47992c3666cc44a7e94c0             2130
2     1f50f920176fa81dab994f9023523100             2017
3     cc419e0650a3c5ba77189a1882b7556a             1893
4     da8622b14eb17ae2831f4ac5b9dab84a             1662
...                                ...              ...
3090  d4828feed4cda2cd9d6b37899a95a71e                1
3091  d4d99b60c4c94cd35acd502cec65dc8d                1
3092  5f57db27027655e6c6a391601daa0258                1
3093  d4f91e8be86b38da2ee35bc489645a9d                1
3094  7d81e74a4755b552267cd5e081563028                1

[3095 rows x 2 columns]


### Top-rated product categories (based on average review score)


In [13]:


top_rated_products = df.groupby('product_category_name')['review_score'].mean().sort_values(ascending=False).reset_index()

top_rated_products.columns = ['Product category name','Review Score']

# Display the result
print(top_rated_products)


                            Product category name  Review Score
0                               cds_dvds_musicais      4.642857
1                   fashion_roupa_infanto_juvenil      4.500000
2                          livros_interesse_geral      4.438503
3                                          flores      4.419355
4                               livros_importados      4.419355
..                                            ...           ...
68                              moveis_escritorio      3.526791
69  portateis_cozinha_e_preparadores_de_alimentos      3.266667
70                                fraldas_higiene      3.256410
71                                       pc_gamer      3.100000
72                             seguros_e_servicos      2.500000

[73 rows x 2 columns]


### Number of sellers by state

In [14]:
sellers_by_state = df.groupby('seller_state')['seller_id'].count().sort_index().reset_index()

sellers_by_state.columns = ['State','Number Of Sellers']

print(sellers_by_state)

   State  Number Of Sellers
0     AC                  1
1     AM                  3
2     BA                700
3     CE                103
4     DF                949
5     ES                390
6     GO                550
7     MA                410
8     MG               9314
9     MS                 60
10    MT                147
11    PA                  8
12    PB                 44
13    PE                465
14    PI                 12
15    PR               9096
16    RJ               5036
17    RN                 56
18    RO                 14
19    RS               2294
20    SC               4271
21    SE                 10
22    SP              84377


### Average review score by seller and product


In [15]:
avg_score_by_seller_product = df.groupby(['product_id'])['review_score'].mean().reset_index()
avg_score_by_seller_product.columns = ['Product ID','Review Score']
print(pd.DataFrame(avg_score_by_seller_product))


                             Product ID  Review Score
0      00066f42aeeb9f3007548bb9d3f33c38           5.0
1      00088930e925c41fd95ebfe695fd2655           4.0
2      0009406fd7479715e4bef61dd91f2462           1.0
3      000b8f95fcb9e0096488278317764d19           5.0
4      000d9be29b5207b54e86aa1b1ac54872           5.0
...                                 ...           ...
32946  fff6177642830a9a94a0f2cba5e476d1           4.5
32947  fff81cc3158d2725c0655ab9ba0f712c           4.0
32948  fff9553ac224cec9d15d49f5a263411f           5.0
32949  fffdb2d0ec8d6a61f0a0a0db3f25b441           5.0
32950  fffe9eeff12fcbd74a2f2b007dde0c58           4.0

[32951 rows x 2 columns]


### Seller with the best average rating


In [16]:


# Count the number of sales per seller
seller_sales = df.groupby('seller_id')['order_id'].count()

# Filter sellers with at least 50 sales
eligible_sellers = seller_sales[seller_sales >= 50].index

# Keep only the rows corresponding to eligible sellers
filtered_df = df[df['seller_id'].isin(eligible_sellers)]

# Compute the average review score per seller
seller_satisfaction = filtered_df.groupby('seller_id')['review_score'].mean()

# Identify the seller with the highest average satisfaction
top_seller = seller_satisfaction.idxmax()
top_score = seller_satisfaction.max()

# Display the result
print(f"Top seller: {top_seller} with average satisfaction: {top_score:.2f}")


Top seller: d9bd94811c3338dceb4181f3dbc0c73e with average satisfaction: 4.82


### Find the seller with the highest average satisfaction per state, among those who had more than 50 orders.


In [17]:


seller_sales = df.groupby('seller_id')['order_id'].count()

data = seller_sales[seller_sales>50].index

filtred_data = df[df['seller_id'].isin(data)]

seller_satisfaction = filtred_data.groupby(['seller_id','seller_state'])['review_score'].mean().sort_values(ascending=False)




seller_satisfaction


seller_id                         seller_state
d9bd94811c3338dceb4181f3dbc0c73e  SP              4.822581
d13e50eaa47b4cbe9eb81465865d8cfc  SP              4.816901
83e197e95a1bbabc8c75e883ed016c47  RJ              4.732143
d566c37fa119d5e66c4e9052e83ee4ea  SP              4.718310
376a891762bbdecbc02b4b6adec3fdda  GO              4.683333
                                                    ...   
54965bbe3e4f07ae045b90b0b8541f52  PR              2.939759
602044f2c16190c2c6e45eb35c2e21cb  SP              2.932203
972d0f9cf61b499a4812cf0bfa3ad3c4  SC              2.931818
2eb70248d66e0e3ef83659f71b244378  SP              2.716279
1ca7077d890b907f89be8c954a02686a  SP              2.189781
Name: review_score, Length: 479, dtype: float64

### Orders never delivered


In [19]:
df[df.order_delivered_customer_date.isna()]

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,review_id,review_score,...,payment_installments,payment_value,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,lat,lng,delivery_time,shipping_month
8,136cce7faa42fdb2cefd53fdc79a6098,ed0271e0b7da060a393796590e7b737a,invoiced,2017-04-11 12:22:08,2017-04-13 13:25:17,NaN,NaN,2017-05-09 00:00:00,e07549ef5311abcc92ba1784b093fb56,2.0,...,1.0,65.95,36edbb3fb164b1f16485364b6fb04c73,98900,santa rosa,RS,-27.865358,-54.470128,NaT,2017-04
51,ee64d42b8cf066f35eac1cf57de1aa85,caded193e8e47b8362864762a83db3c5,shipped,2018-06-04 16:44:48,2018-06-05 04:31:18,2018-06-05 14:32:00,NaN,2018-06-28 00:00:00,5853e27c64b89e222c5e4edf78bec3d4,1.0,...,1.0,22.36,08fb46d35bb3ab4037202c23592d1259,13215,jundiai,SP,-23.169668,-46.886715,NaT,2018-06
125,0760a852e4e9d89eb77bf631eaaf1c84,d2a79636084590b7465af8ab374a8cf5,invoiced,2018-08-03 17:44:42,2018-08-07 06:15:14,NaN,NaN,2018-08-21 00:00:00,c4b17a374b71539eb360fe5baaa9a10e,3.0,...,1.0,50.35,c7f8d7b1fffc946d7069574f74c39f4e,88140,santo amaro da imperatriz,SC,-27.687415,-48.771889,NaT,2018-08
155,15bed8e2fec7fdbadb186b57c46c92f2,f3f0e613e0bdb9c7cee75504f0f90679,processing,2017-09-03 14:22:03,2017-09-03 14:30:09,NaN,NaN,2017-10-03 00:00:00,8079a046b23b88c4cea485cfbd8aec39,5.0,...,2.0,138.28,9f269af9c49244f6ba4a46985a3cfc2e,3436,sao paulo,SP,-23.559514,-46.531410,NaT,2017-09
192,6942b8da583c2f9957e990d028607019,52006a9383bf149a4fb24226b173106f,shipped,2018-01-10 11:33:07,2018-01-11 02:32:30,2018-01-11 19:39:23,NaN,2018-02-07 00:00:00,NaN,NaN,...,1.0,69.12,528b011eb7fab3d59c336cc7248eed3a,38600,paracatu,MG,-17.224179,-46.874265,NaT,2018-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118946,3a3cddda5a7c27851bd96c3313412840,0b0d6095c5555fe083844281f6b093bb,canceled,2018-08-31 16:13:44,NaN,NaN,NaN,2018-10-01 00:00:00,16d3db296c46b4411dfc5f8bde882e6d,5.0,...,1.0,63.89,e90598185d2427a35e32ef241a5c04aa,11075,santos,SP,-23.954600,-46.337731,NaT,NaT
118995,e9e64a17afa9653aacf2616d94c005b8,b4cd0522e632e481f8eaf766a2646e86,processing,2018-01-05 23:07:24,2018-01-09 07:18:05,NaN,NaN,2018-02-06 00:00:00,f1f9860e77fbdbd7cfcfd9a94e0cf2ef,1.0,...,1.0,144.56,965191786b70912f793e4a777fd623cd,13050,campinas,SP,-22.940383,-47.095263,NaT,2018-01
119039,a89abace0dcc01eeb267a9660b5ac126,2f0524a7b1b3845a1a57fcf3910c4333,canceled,2018-09-06 18:45:47,NaN,NaN,NaN,2018-09-27 00:00:00,047fd109ced39e02296f6aeb74f6a6f1,4.0,...,1.0,107.13,d05c44a138277ad325d915c6b7ccbcdf,5344,sao paulo,SP,-23.553025,-46.745244,NaT,NaT
119040,a69ba794cc7deb415c3e15a0a3877e69,726f0894b5becdf952ea537d5266e543,unavailable,2017-08-23 16:28:04,2017-08-28 15:44:47,NaN,NaN,2017-09-15 00:00:00,41b9de18e0dd1e085e3c402d1c9b87d0,1.0,...,2.0,195.91,e72a90a2b29fe1a8795b284aaaa3246f,22723,rio de janeiro,RJ,-22.916897,-43.402789,NaT,NaT


### Canceled Order

In [20]:
df[df['order_status'] == 'canceled']

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,review_id,review_score,...,payment_installments,payment_value,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,lat,lng,delivery_time,shipping_month
470,1b9ecfe83cdc259250e1a8aca174f0ad,6d6b50b66d79f80827b6d96751528d30,canceled,2018-08-04 14:29:27,2018-08-07 04:10:26,NaN,NaN,2018-08-14 00:00:00,14d540de5632307e389e9416086fc978,1.0,...,1.0,33.34,8ea097b1824dbd4d17af71b0afe04301,5514,sao paulo,SP,-23.584007,-46.687883,NaT,2018-08
737,714fb133a6730ab81fa1d3c1b2007291,e3fe72696c4713d64d3c10afe71e75ed,canceled,2018-01-26 21:34:08,2018-01-26 21:58:39,2018-01-29 22:33:25,NaN,2018-02-22 00:00:00,f242ce44c6572a94907a446b8fda32c8,1.0,...,4.0,96.01,c4ebedb09beb89cc0314c5c0c33f8053,9961,diadema,SP,-23.702465,-46.596052,NaT,2018-02
738,714fb133a6730ab81fa1d3c1b2007291,e3fe72696c4713d64d3c10afe71e75ed,canceled,2018-01-26 21:34:08,2018-01-26 21:58:39,2018-01-29 22:33:25,NaN,2018-02-22 00:00:00,105facb42b5b9147996f0b1b6fbf7b80,1.0,...,4.0,96.01,c4ebedb09beb89cc0314c5c0c33f8053,9961,diadema,SP,-23.702465,-46.596052,NaT,2018-02
1272,3a129877493c8189c59c60eb71d97c29,0913cdce793684e52bbfac69d87e91fd,canceled,2018-01-25 13:34:24,2018-01-25 13:50:20,2018-01-26 21:42:18,NaN,2018-02-23 00:00:00,15c8f3b3ff412d0f8d6b72053335fd0b,1.0,...,1.0,14.12,11c3d6c93dea6ce86769a4835c171bd9,87013,maringa,PR,-23.422486,-51.938405,NaT,2018-01
1273,3a129877493c8189c59c60eb71d97c29,0913cdce793684e52bbfac69d87e91fd,canceled,2018-01-25 13:34:24,2018-01-25 13:50:20,2018-01-26 21:42:18,NaN,2018-02-23 00:00:00,15c8f3b3ff412d0f8d6b72053335fd0b,1.0,...,1.0,36.88,11c3d6c93dea6ce86769a4835c171bd9,87013,maringa,PR,-23.422486,-51.938405,NaT,2018-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118476,e49e7ce1471b4693482d40c2bd3ad196,e4e7ab3f449aeb401f0216f86c2104db,canceled,2018-08-07 11:16:28,NaN,NaN,NaN,2018-08-10 00:00:00,68c3385b4bb41af5847346c3552ba744,2.0,...,1.0,54.42,13d7bd0b05e18b00f415221b3bb69348,9810,sao bernardo do campo,SP,-23.719576,-46.564753,NaT,NaT
118788,6560fb10610771449cb0463c5ba12199,0d07d0a588caf93cc66b7a8aff86d2fe,canceled,2017-10-01 22:26:25,2017-10-01 22:35:22,NaN,NaN,2017-10-27 00:00:00,da12d47f5616024c216d6a5ad19ec5ac,1.0,...,3.0,333.62,0f75924cbbe60142068f4029d744b724,82310,curitiba,PR,-25.422300,-49.358726,NaT,2017-10
118789,6560fb10610771449cb0463c5ba12199,0d07d0a588caf93cc66b7a8aff86d2fe,canceled,2017-10-01 22:26:25,2017-10-01 22:35:22,NaN,NaN,2017-10-27 00:00:00,da12d47f5616024c216d6a5ad19ec5ac,1.0,...,3.0,333.62,0f75924cbbe60142068f4029d744b724,82310,curitiba,PR,-25.422300,-49.358726,NaT,2017-10
118946,3a3cddda5a7c27851bd96c3313412840,0b0d6095c5555fe083844281f6b093bb,canceled,2018-08-31 16:13:44,NaN,NaN,NaN,2018-10-01 00:00:00,16d3db296c46b4411dfc5f8bde882e6d,5.0,...,1.0,63.89,e90598185d2427a35e32ef241a5c04aa,11075,santos,SP,-23.954600,-46.337731,NaT,NaT


### Possible returns

In [21]:

possible_returns = df[
    (df['order_status'] == 'delivered') &
    (df['review_score'].isin([1,2]))
]
possible_returns

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,review_id,review_score,...,payment_installments,payment_value,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,lat,lng,delivery_time,shipping_month
10,76c6e866289321a7c93b82b54852dc33,f54a9f0e6b351c431402b8461ea51999,delivered,2017-01-23 18:29:09,2017-01-25 02:50:47,2017-01-26 14:16:31,2017-02-02 14:08:10,2017-03-06 00:00:00,fc4af8aea8ec3f1a3cd181d3d0cadbd5,1.0,...,1.0,35.95,39382392765b6dc74812866ee5ee92a7,99655,faxinalzinho,RS,-27.421769,-52.675022,9 days 19:39:01,2017-01
13,e6ce16cb79ec1d90b1da9085a6118aeb,494dded5b201313c64ed7f100595b95c,delivered,2017-05-16 19:41:10,2017-05-16 19:50:18,2017-05-18 11:40:40,2017-05-29 11:18:31,2017-06-07 00:00:00,15898b543726a832d4137fbef5d1d00e,1.0,...,1.0,259.06,f2a85dec752b8517b5e58a06ff3cd937,20780,rio de janeiro,RJ,-22.896463,-43.272172,12 days 15:37:21,2017-05
14,e6ce16cb79ec1d90b1da9085a6118aeb,494dded5b201313c64ed7f100595b95c,delivered,2017-05-16 19:41:10,2017-05-16 19:50:18,2017-05-18 11:40:40,2017-05-29 11:18:31,2017-06-07 00:00:00,15898b543726a832d4137fbef5d1d00e,1.0,...,1.0,259.06,f2a85dec752b8517b5e58a06ff3cd937,20780,rio de janeiro,RJ,-22.896463,-43.272172,12 days 15:37:21,2017-05
26,203096f03d82e0dffbc41ebc2e2bcfb7,d2b091571da224a1b36412c18bc3bbfe,delivered,2017-09-18 14:31:30,2017-09-19 04:04:09,2017-10-06 17:50:03,2017-10-09 22:23:46,2017-09-28 00:00:00,38cae21b1b57a95959440380d5b2ef7a,2.0,...,1.0,118.86,d699688533772c15a061e8ce81cb56df,4001,sao paulo,SP,-23.572939,-46.651115,21 days 07:52:16,2017-09
31,fbf9ac61453ac646ce8ad9783d7d0af6,3a874b4d4c4b6543206ff5d89287f0c3,delivered,2018-02-20 23:46:53,2018-02-22 02:30:46,2018-02-26 22:25:22,2018-03-21 22:03:54,2018-03-12 00:00:00,6a1a8e54de03ab98e6e8ff56e56e507f,2.0,...,1.0,125.43,a25d5f94840d3c6a1a49f271ed83f4ec,21715,rio de janeiro,RJ,-22.879477,-43.440264,28 days 22:17:01,2018-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119120,38e9133ce29f6bbe35aed9c3863dce01,ad312389a098ceff46ce92c4595c06d0,delivered,2017-10-12 20:54:11,2017-10-14 03:28:24,2017-10-17 17:04:42,2017-11-21 17:06:59,2017-10-31 00:00:00,4a57b77d844594f24904a568218a2d96,1.0,...,1.0,48.75,a3983b8b6b679c9f645e6ef9517ac98d,14802,araraquara,SP,-21.772226,-48.180222,39 days 20:12:48,2017-10
119122,d692ef54145c9cb3322ec2e5508aa3f4,82ddfcf9438b0cd1117b55ac33184df8,delivered,2018-03-21 19:47:18,2018-03-21 20:05:26,2018-03-22 21:11:58,2018-04-11 00:48:31,2018-04-09 00:00:00,6b2ee488cd87d8a9ee67eb63ad5a7a4c,1.0,...,2.0,146.83,397cd1affbb66fa8f76e8ac24685440a,13272,valinhos,SP,-22.951602,-47.001996,20 days 05:01:13,2018-03
119129,cfa78b997e329a5295b4ee6972c02979,a2f7428f0cafbc8e59f20e1444b67315,delivered,2017-12-20 09:52:41,2017-12-20 10:09:52,2017-12-20 20:25:25,2018-01-26 15:45:14,2018-01-18 00:00:00,8ea780895f362ce949029f95b7181ff0,1.0,...,1.0,71.04,a49e8e11e850592fe685ae3c64b40eca,83870,campo do tenente,PR,-25.985590,-49.678273,37 days 05:52:33,2017-12
119140,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15 00:00:00,8ab6855b9fe9b812cd03a480a25058a1,2.0,...,4.0,441.16,5097a5312c8b157bb7be58ae360ef43c,28685,japuiba,RJ,-22.562825,-42.694574,17 days 02:04:27,2018-01
